# Developing an AI application

This is my solution for the pytorch challenge project.

In this project, I'll train an image classifier to recognize different species of flowers. We'll be using <a href="http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html">this</a> dataset of 102 flower categories.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

train_on_gpu = torch.cuda.is_available()
if train_on_gpu:
    print("Cuda is available. Training on GPU...")
else:
    print("Cuda is not available. Training on CPU...")

# Loading data
The dataset I'll be using can be downloaded <a href="https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip">here</a>. As you can see in the files, the dataset already provided us with a training and validation set.

I'll be using a pre-trained network (resnet152) so we'll need to normalize the means and standard deviations of the images to what the network expects which is trained on ImageNet dataset. For the means, it's ```[0.485, 0.456, 0.406]``` and for the standard deviations ```[0.229, 0.224, 0.225]```, calculated from the ImageNet images. These values will shift each color channel to be centered at 0 and range from -1 to 1. In addition, our input data has to be of ```224x224``` pixels required by the network.

In [ ]:
# Define our dataset dir
data_dir = 'flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

In [ ]:
import torchvision.datasets as datasets
# Transforms for the training and validation sets
train_transforms = transforms.Compose([
    transforms.RandomRotation(40),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

validate_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
validate_data = datasets.ImageFolder(valid_dir, transform=validate_transforms)

batch_size=32

# Using the image datasets and the trainforms, define the dataloaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
validate_loader = torch.utils.data.DataLoader(validate_data, batch_size=batch_size, shuffle=True)

# Define our neutral network and classifier
I decided to pick resnet152 as their CNN seems suited for our use-case.
I've also tried training with densenet201 and densenet161 networks but resnet152 gave me the best validate and test accuracy

You can learn more about it <a href="https://arxiv.org/abs/1512.03385">here</a>.

In [ ]:
# Build network
import torchvision.models as models
import torch.nn as nn

# Pre-trained model
model = models.resnet152(pretrained=True)

# Freeze the parameters in the conv layers
for param in model.parameters():
    param.requires_grad = False

# construct a new classifier for fc layer
classifier = nn.Sequential(nn.Linear(2048, 512),
                           nn.ReLU(),
                           nn.Dropout(p=0.25),
                           nn.Linear(512, 102))

# set our classifier to the fc layer
model.fc = classifier

# move tensors to GPU if CUDA is available
if train_on_gpu:
    model.cuda()

print(model)

In [ ]:
# loss function
criterion = nn.CrossEntropyLoss()

# optimizer
import torch.optim as optim
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=8, verbose=True)

# Training and saving the network
We'll be saving our network(state_dict) everytime when our validation loss reaches a new minimum while it's training.

In [ ]:
# train the network

# number of epochs to train the model
n_epochs = 70

valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs + 1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0

    ###################
    # train the model #
    ###################
    model.train()
    for data, target in train_loader:
        # move tensors to GPU if CUDA is available
        data, target = data.cuda(), target.cuda()
        
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item() * data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for data, target in validate_loader:
        # move tensors to GPU if CUDA is available
        data, target = data.cuda(), target.cuda()
        
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item() * data.size(0)
    
    # update our learning rate if it's reaches a condition set by scheduler
    scheduler.step(valid_loss)
    
    # calculate average losses
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(validate_loader.dataset)

    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))

    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_divit_pytorch_challenge.pt')
        valid_loss_min = valid_loss
        

# Testing the model
I'll be using https://github.com/GabrielePicco/deep-learning-flower-identifier class methods to test our model.

In [ ]:
import os
import random
import sys
import urllib.request
import zipfile

def calc_accuracy(model, input_image_size, use_google_testset=False, testset_path=None, batch_size=32,
                  norm_mean=[0.485, 0.456, 0.406], norm_std=[0.229, 0.224, 0.225]):
    """
    Calculate the mean accuracy of the model on the test test
    :param use_google_testset: If true use the testset derived from google image
    :param testset_path: If None, use a default testset (missing image from the Udacity dataset,
    downloaded from here: http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz)
    :param batch_size:
    :param model:
    :param input_image_size:
    :param norm_mean:
    :param norm_std:
    :return: the mean accuracy
    """
    if use_google_testset:
        testset_path = "./google_test_data"
        url = 'https://www.dropbox.com/s/3zmf1kq58o909rq/google_test_data.zip?dl=1'
        download_test_set(testset_path, url)
    if testset_path is None:
        testset_path = "./flower_data_orginal_test"
        url = 'https://www.dropbox.com/s/da6ye9genbsdzbq/flower_data_original_test.zip?dl=1'
        download_test_set(testset_path, url)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.eval()
    model.to(device=device)
    with torch.no_grad():
        batch_accuracy = []
        torch.manual_seed(33)
        torch.cuda.manual_seed(33)
        np.random.seed(33)
        random.seed(33)
        torch.backends.cudnn.deterministic = True
        datatransform = transforms.Compose([transforms.RandomRotation(45),
                                            transforms.Resize(input_image_size + 32),
                                            transforms.CenterCrop(input_image_size),
                                            transforms.RandomHorizontalFlip(),
                                            transforms.ToTensor(),
                                            transforms.Normalize(norm_mean, norm_std)])
        image_dataset = datasets.ImageFolder(testset_path, transform=datatransform)
        dataloader = torch.utils.data.DataLoader(image_dataset, batch_size=batch_size, shuffle=True, worker_init_fn=_init_fn)
        for idx, (inputs, labels) in enumerate(dataloader):
            if device == 'cuda':
                inputs, labels = inputs.cuda(), labels.cuda()
            outputs = model.forward(inputs)
            _, predicted = outputs.max(dim=1)
            equals = predicted == labels.data
            print("Batch accuracy (Size {}): {}".format(batch_size, equals.float().mean()))
            batch_accuracy.append(equals.float().mean().cpu().numpy())
        mean_acc = np.mean(batch_accuracy)
        print("Mean accuracy: {}".format(mean_acc))
    return mean_acc

def download_test_set(default_path, url):
    """
    Download a testset containing approximately 10 images for every flower category.
    The images were download with the download_testset script and hosted on dropbox.
    :param default_path:
    :return:
    """
    if not os.path.exists(default_path):
        print("Downloading the dataset from: {}".format(url))
        tmp_zip_path = "./tmp.zip"
        urllib.request.urlretrieve(url, tmp_zip_path, download_progress)
        with zipfile.ZipFile(tmp_zip_path, 'r') as zip_ref:
            zip_ref.extractall(default_path)
        os.remove(tmp_zip_path)

def download_progress(blocknum, blocksize, totalsize):
    """
    Show download progress
    :param blocknum:
    :param blocksize:
    :param totalsize:
    :return:
    """
    readsofar = blocknum * blocksize
    if totalsize > 0:
        percent = readsofar * 1e2 / totalsize
        s = "\r%5.1f%% %*d / %d" % (
            percent, len(str(totalsize)), readsofar, totalsize)
        sys.stderr.write(s)
        if readsofar >= totalsize: # near the end
            sys.stderr.write("\n")
    else: # total size is unknown
        sys.stderr.write("read %d\n" % (readsofar,))

def _init_fn(worker_id):
    """
    It makes determinations applied transforms
    :param worker_id:
    :return:
    """
    np.random.seed(77 + worker_id)

In [ ]:
# testing
calc_accuracy(model, 224)

# Label mapping
This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers. The json file can be found in this project directory.

In [ ]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Save checkpoint
Here we could save more information about our model if we need to continue to train our model.

In [ ]:
# Save the additional state
model.class_to_idx = train_data.class_to_idx

checkpoint = {'class_to_idx' : model.class_to_idx,
              'optimizer_state_dict' : optimizer.state_dict(),
              'cat_to_name' : cat_to_name,
              'classifier' : classifier,
              'epoch' : n_epochs,
              'state_dict' : model.state_dict()}

torch.save(checkpoint, 'pytorch_final_divit_checkpoint.pt')

# Loading checkpoint
Here is how I would load my model if I need it.

In [ ]:
def load_my_model(filePath):
    checkpoint = torch.load(filePath)
    model = models.resnet152(pretrained=True)
    model.fc = checkpoint['classifier']
    model.load_state_dict(checkpoint['state_dict'])
    return model
